In [32]:
import pandas as pd
import re
from datasets import Dataset

# Load Dataset

In [3]:
data_path = "../data/naruto.csv"
naruto_transcript_df = pd.read_csv(data_path)
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [5]:
# Remove actions from transcript
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    return result

In [6]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [15]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


0      [Give, it, up., , You’re, just, bent,, because...
1      [I, hope, you’re, not, bothering, me, with, so...
2                                        [Naseer, Sabah]
3                     [is, the, best, person, on, earth]
4                                                 [muah]
                             ...                        
158    [Congratulations., You, graduate., Naruto’s, s...
159                                               [Huh?]
160                                     [Iruka, Sensei!]
161                                  [Ah!, That, hurts!]
162    [Naruto., This, is, only, the, beginning., The...
Name: line, Length: 163, dtype: object

In [21]:
# Choose naruto character to train chatbot on, and filter short lines
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].str.strip().str.split(" ")
naruto_transcript_df['number_of_words'] = naruto_transcript_df['number_of_words'].apply(lambda x: len(x))
 

In [22]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [23]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name'] == 'Naruto')&(naruto_transcript_df['number_of_words'] > 5), 'naruto_response_flag'] = 1

In [24]:
naruto_transcript_df.head()

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0


In [25]:
# indices of naruto responses are where naruto_response_flag is 1 (exept first sentence, not a response but an initial statement)
indexes_to_take = list(naruto_transcript_df[naruto_transcript_df['naruto_response_flag'] == 1].index)[1:]

In [27]:
indexes_to_take[:3]

[6, 28, 30]

In [28]:
system_prompt = """ You are Neruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n"""
prompts = []
for index in indexes_to_take:
    prompt = system_prompt
    prompt += naruto_transcript_df.iloc[index-1]['line'] # the sentence that naruto responded to
    prompt += "\n"
    prompt += naruto_transcript_df.iloc[index]['line']
    prompts.append(prompt)
    

In [30]:
print(prompts[4])

 You are Neruto from the anime "Naruto". Your responses should reflect his personality and speech patterns 
 Why would you do that to the Hokage faces? I mean, you know who the Hokage are, don’t you?
 Course I do.  Everybody knows. They were the greatest Shinobi of their time, right? The best of the best. Undefeated Ninja champs.   And the Fourth Hokage’s the one who saved the village from the Nine-Tailed Fox. He was the most amazing.


In [31]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,"You are Neruto from the anime ""Naruto"". Your ..."
1,"You are Neruto from the anime ""Naruto"". Your ..."
2,"You are Neruto from the anime ""Naruto"". Your ..."
3,"You are Neruto from the anime ""Naruto"". Your ..."
4,"You are Neruto from the anime ""Naruto"". Your ..."


In [33]:
dataset = Dataset.from_pandas(df)